In [3]:
# Unsloth 라이브러리에서 FastLanguageModel을 임포트
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # 최대 시퀀스 길이를 설정 ( 텍스트의 최대 길이를 지정)
dtype = None  # 자동 감지를 위해 None 설정. Tesla T4는 Float16, Ampere+는 Bfloat16 사용. 모델의 파라미터를 저장할 데이터 타입
load_in_4bit = True  # 메모리 사용량을 줄이기 위해 4비트 양자화 사용. 다만 양자화에 따른 손실이 있어서 필요에 따라 False로 설정 가능

# 사전 학습된 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",  # 사용할 모델 이름
    max_seq_length = max_seq_length,         # 설정한 최대 시퀀스 길이
    dtype = dtype,                           # 데이터 타입 설정
    load_in_4bit = load_in_4bit,             # 4비트 양자화 여부
)

This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/alpaco/anaconda3/envs/fast_langch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
# PEFT(파라미터 효율적 파인튜닝) 모델 설정
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA 랭크 설정. 8, 16, 32, 64, 128 권장. r 값이 클수록 모델이 더 많은 정보를 학습할 수 있지만, 너무 크면 메모리를 많이 사용
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],  # PEFT 적용할 모듈 목록. 모델의 특정 부분(모듈)에만 학습
    lora_alpha = 16,        # LoRA 알파 설정 LoRA라는 기술이 얼마나 강하게 작용할지 조절
    lora_dropout = 0,       # LoRA 드롭아웃 설정. 0으로 최적화
    bias = "none",          # 바이어스 설정. "none"으로 최적화

    # "unsloth" 사용 시 VRAM 절약 및 배치 사이즈 2배 증가
    # 학습할 때 메모리를 절약하는 방법을 사용하는 설정
    use_gradient_checkpointing = "unsloth",  # 매우 긴 컨텍스트를 위해 "unsloth" 설정
    random_state = 3407,    # 랜덤 시드 설정
    use_rslora = False,     # 랭크 안정화 LoRA 사용 여부
    loftq_config = None,    # LoftQ 설정 (사용하지 않음)
)

Unsloth 2025.6.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
# 모델에게 주어질 텍스트의 형식을 정의
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# 맨윗줄: 모델에게 앞으로 제공될 지시사항을 기반으로 적절한 응답을 작성하라고 말함.
# 모델이 어떤 작업을 수행해야 하는지 명확히 이해할 수 있도록 도와줌
# Instruction:은 지시사항이 제공될 부분
# Response:는 모델이 생성해야 할 응답이 제공될 부분

# EOS 토큰 가져오기 (생성 종료를 위해 필요)
EOS_TOKEN = tokenizer.eos_token  # 반드시 EOS_TOKEN을 추가해야 함
EOS_TOKEN

'<|end_of_text|>'

In [6]:
# 프롬프트 포맷팅 함수 정의
def formatting_prompts_func(examples):
    instructions = examples["유저"]  # 데이터셋의 'instruction' 필드
    outputs      = examples["챗봇"]       # 데이터셋의 'output' 필드
    texts = []
    for instruction, output in zip(instructions, outputs):
                                                           # EOS_TOKEN을 추가하지 않으면 생성이 무한히 계속됨
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN  # 프롬프트 형식에 맞게 텍스트 생성
        texts.append(text)
    return { "text" : texts, }  # 'text' 필드로 반환

#from datasets import load_dataset  # Hugging Face datasets 라이브러리 임포트

# 데이터셋 로드 (Teddy Lee의 QA 데이터셋 미니 버전)
#dataset = load_dataset("teddylee777/QA-Dataset-mini", split = "train")

# 프롬프트 포맷팅 함수 적용하여 데이터셋 변환
#dataset = dataset.map(formatting_prompts_func, batched = True,)

from datasets import load_dataset
dataset = load_dataset( "csv", data_files = "/home/alpaco/chat_bot/wellnis.csv", split = "train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# 예제 엑셀 형태로 데이터셋을 만든다면?
# from datasets import load_dataset
# dataset = load_dataset( "csv", data_files = "data.csv", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched=True)
#
#

In [7]:
dataset['text'][:5]

['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n다 제 잘못이에요.\n\n### Response:\n낚시를 하듯이 조금 더 먼 곳을 보고, 조금 더 마음을 비우면 편안해질 거예요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n어렸을 때부터 엄마보다는 아빠랑 더 친했어요.\n\n### Response:\n그러시군요. 아버지에 대해 더 들려주세요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n몸무게도 늘고요.\n\n### Response:\n체중이 단기간에 많이 늘어나면 건강에 좋지 않다고 해요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n눈물이 날 거 같더라고.\n\n### Response:\n감수성이 풍부하다는 증거 같아요. 눈물이 많은 사람 중에 나쁜 사람은 없잖아요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n회사를 옮기면서

In [8]:
## 학습 설정
from trl import SFTTrainer  # TRL 라이브러리에서 SFTTrainer 임포트
from transformers import TrainingArguments  # 트랜스포머 라이브러리에서 TrainingArguments 임포트
from unsloth import is_bfloat16_supported  # BFloat16 지원 여부 확인 함수 임포트

# SFTTrainer 인스턴스 생성
trainer = SFTTrainer(
    model = model,                           # 학습할 모델
    tokenizer = tokenizer,                   # 사용할 토크나이저
    train_dataset = dataset,                 # 학습할 데이터셋 ★★★★★★★★
    dataset_text_field = "text",             # 데이터셋의 텍스트 필드 이름 ★★★★★★★★
    max_seq_length = max_seq_length,         # 최대 시퀀스 길이
    dataset_num_proc = 2,                    # 데이터셋 전처리에 사용할 프로세스 수 cpu
    packing = False,                         # 짧은 시퀀스의 경우 packing을 비활성화 (학습 속도 5배 향상 가능)
    args = TrainingArguments(
        per_device_train_batch_size = 2,     # 디바이스 당 배치 사이즈
        gradient_accumulation_steps = 4,     # 그래디언트 누적 단계 수
        warmup_steps = 5,                     # 워밍업 스텝 수
        # num_train_epochs = 1,               # 전체 학습 에폭 수 설정 가능
        max_steps = 60,                       # 최대 학습 스텝 수
        learning_rate = 2e-4,                 # 학습률
        fp16 = not is_bfloat16_supported(),   # BFloat16 지원 여부에 따라 FP16 사용
        bf16 = is_bfloat16_supported(),       # BFloat16 사용 여부
        logging_steps = 1,                    # 로깅 빈도
        optim = "adamw_8bit",                  # 옵티마이저 설정 (8비트 AdamW)
        weight_decay = 0.01,                  # 가중치 감쇠
        lr_scheduler_type = "linear",         # 학습률 스케줄러 타입
        seed = 3407,                           # 랜덤 시드 설정
        output_dir = "outputs",                # 출력 디렉토리
    ),
)

## 학습 실행
trainer_stats = trainer.train()  # 모델 학습 시작 # 3d6e4b9a0ddcd9edfafad59dcde0fd8c666954fd

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 857 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,3.270500
2,3.395300
3,3.409000
4,3.290200
5,3.063500
6,2.566400
7,2.514000
8,2.319900
9,2.020200
10,1.875400


In [9]:
# 모델 저장 로컬폴더에다가 저장하는 방식
model.save_pretrained("wellnis")  # Local saving
tokenizer.save_pretrained("wellnis")

# 이 이외에 허깅페이스나 다른 hub에 push해서 저장하는 방법이 있음
# 다만, 업로드 속도와 다운로드 속도를 고려해야함.

('wellnis/tokenizer_config.json',
 'wellnis/special_tokens_map.json',
 'wellnis/tokenizer.json')

In [10]:
from unsloth import FastLanguageModel
import torch

# 저장된 경로 지정
save_directory = "/home/alpaco/chat_bot/wellnis"

# 모델과 토크나이저 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_directory,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,  # 양자화 옵션을 동일하게 설정
)


==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [12]:
# ...기존 모델/토크나이저 로딩 코드 이후...

from transformers import TextStreamer

# 1. 투샷 프롬프트 정의
two_shot_prompt = """
### Instruction:
너무 무기력해.

### Response:
무기력증은 삶의 목적과 의미를 잃고, 아무 일도 하지 못하는 상태를 말합니다. 운동요법은 신체 활동을 통해 무기력증을 완화시키는 데 도움이 됩니다.

### Instruction:
무기력증 치료법을 알려줘.

### Response:
무기력증은 심리치료, 약물치료, 운동요법 등 다양한 방법으로 치료할 수 있습니다.

### Instruction:
운동이 도움이 될까?

### Response:
"""

In [13]:
# 2. 추론을 위한 입력 준비
inputs = tokenizer(
    two_shot_prompt,
    return_tensors="pt"
).to("cuda")

# 3. TextStreamer 초기화
text_streamer = TextStreamer(tokenizer)

# 4. 모델을 사용하여 텍스트 생성 및 스트리밍 출력
output = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

<|begin_of_text|>
### Instruction:
너무 무기력해.

### Response:
무기력증은 삶의 목적과 의미를 잃고, 아무 일도 하지 못하는 상태를 말합니다. 운동요법은 신체 활동을 통해 무기력증을 완화시키는 데 도움이 됩니다.

### Instruction:
무기력증 치료법을 알려줘.

### Response:
무기력증은 심리치료, 약물치료, 운동요법 등 다양한 방법으로 치료할 수 있습니다.

### Instruction:
운동이 도움이 될까?

### Response:
운동은 신체 활동을 통해 몸과 마음을 건강하게 해줍니다.<|end_of_text|>


In [14]:
# 5. 답변만 추출
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text.split("### Response:")[-1].strip()

'운동은 신체 활동을 통해 몸과 마음을 건강하게 해줍니다.'

# 랭체인: lib

In [ ]:
# langchain-community: 애플리케이션을 개발할때 쓰이는 각종 기능들을 담고 있다,
# ollama: lamma3나 다른 llm들을 다운로드 받고 쓸 수 있는 라이브러리
# chromadb: 청킹 vector들을 저장하는 sql 종류
# PyPDF2: pdf에서 글자를 추출해주는 라이브러리 이거 말고도 OCR로도 추출 가능(클로바ocr, 구글ocr, AI ocr모델 등)
# !pip install --upgrade langchain langchain-community ollama chromadb PyPDF2
# !pip install beautifulsoup4

In [44]:
# 웹 방식
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 감정
url = 'https://ko.wikipedia.org/wiki/%EA%B0%90%EC%A0%95'
loader = WebBaseLoader(url) # url을 셋팅

# 웹페이지 텍스트 -> Documents
docs = loader.load() # load 내장함수로 내용들을 불러옴

# 내용출력
print(len(docs)) # 문서의 길이
print(len(docs[0].page_content)) # 첫번째 문서의 내용
print(docs[0].page_content[1000:2000]) # 첫번째 문서에서 1000: 2000 구간슬라이싱


# Text Split (Documents -> small chunks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 청킹 작업                           # 문장 토큰을 1000길이로 짜름(청킹한다라는뜻), overlap은 끝에 몇글자가 중복되도록 짜를거냐
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs) # 스플리터로 docs 들을 청킹함

print(len(splits))
print(splits[2])


# PDF 파일 방식
# uploaded = files.upload()
# pdf_file = next(iter(uploaded))
# print(f"업로드된 파일: {pdf_file}")

# # PDF 텍스트 추출 및 분할
# try:
#     loader = PyPDFLoader(pdf_file)
#     docs = loader.load()
# except Exception as e:
#     print(f"PDF 로딩 중 오류 발생: {e}")
#     exit()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

1
4452
iRomânăРусскийСаха тылаᱥᱟᱱᱛᱟᱲᱤScotsسنڌيSlovenčinaShqipСрпски / srpskiSvenskaతెలుగుТоҷикӣไทยУкраїнськаOʻzbekcha / ўзбекчаTiếng ViệtWinaray吴语中文粵語

링크 편집











문서토론





한국어

















읽기편집역사 보기







도구





도구
사이드바로 이동
숨기기



		동작
	


읽기편집역사 보기





		일반
	


여기를 가리키는 문서가리키는 글의 최근 바뀜파일 올리기고유 링크문서 정보이 문서 인용하기축약된 URL 얻기QR코드 다운로드





		인쇄/내보내기
	


책 만들기PDF로 다운로드인쇄용 판





		다른 프로젝트
	


위키미디어 공용위키데이터 항목





















보이기
사이드바로 이동
숨기기










위키백과, 우리 모두의 백과사전.


 느낌은 여기로 연결됩니다. 다른 뜻에 대해서는 느낌 (동음이의) 문서를 참고하십시오.
  법률에 대해서는 감정 (법률) 문서를, 감미료에 대해서는 사카린 문서를 참고하십시오.
기본적인 감정의 예.
만화적으로 표현된 일부 감정.
감정(感情) 또는 느낌은 사람이 오감이 아닌 다른 방식으로 느끼는 것으로, 기쁨(희), 노여움(노), 슬픔(애), 즐거움(락) 등이 있다.


역사[편집]
감정에 대한 현대적 개념은 19세기 빌헬름 분트(Wilhelm Wundt)와 함께 발전했다.
사회적, 심리적 정서적 선호(즉, 사람들이 좋아하는 것과 싫어하는 것)에 대한 연구에서 많은 실험이 수행되었다. 선호도, 태도, 인상 형성 및 의사 결정에 대한 구체적인 연구가 수행되었다. 이 연구는 결과를 인식 기억(기존-신판단)과 대조하여 연구자들이 둘 사이의 신뢰할 수 있는 구별을 입증할 수 있도록 한다. 정서 기반 판단과 인지 과정은 지적된 차이점으로 조사되었으며, 일부에서는 정서와 인지가 다양한 방식으로 서로 영향을 미칠 

In [45]:
# page_content 속성
splits[2].page_content

'읽기편집역사 보기\n\n\n\n\n\n\n\n도구\n\n\n\n\n\n도구\n사이드바로 이동\n숨기기\n\n\n\n\t\t동작\n\t\n\n\n읽기편집역사 보기\n\n\n\n\n\n\t\t일반\n\t\n\n\n여기를 가리키는 문서가리키는 글의 최근 바뀜파일 올리기고유 링크문서 정보이 문서 인용하기축약된 URL 얻기QR코드 다운로드\n\n\n\n\n\n\t\t인쇄/내보내기\n\t\n\n\n책 만들기PDF로 다운로드인쇄용 판\n\n\n\n\n\n\t\t다른 프로젝트\n\t\n\n\n위키미디어 공용위키데이터 항목\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n보이기\n사이드바로 이동\n숨기기\n\n\n\n\n\n\n\n\n\n\n위키백과, 우리 모두의 백과사전.\n\n\n 느낌은 여기로 연결됩니다. 다른 뜻에 대해서는 느낌 (동음이의) 문서를 참고하십시오.\n  법률에 대해서는 감정 (법률) 문서를, 감미료에 대해서는 사카린 문서를 참고하십시오.\n기본적인 감정의 예.\n만화적으로 표현된 일부 감정.\n감정(感情) 또는 느낌은 사람이 오감이 아닌 다른 방식으로 느끼는 것으로, 기쁨(희), 노여움(노), 슬픔(애), 즐거움(락) 등이 있다.'

In [46]:
splits[2].metadata

{'source': 'https://ko.wikipedia.org/wiki/%EA%B0%90%EC%A0%95',
 'title': '감정 - 위키백과, 우리 모두의 백과사전',
 'language': 'ko'}

In [ ]:
#pip install langchain-openai
#pip install sentence-transformers

In [ ]:
# import os
# os.environ["OPENAI_API_KEY"] = "API키"

In [47]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# 무료 임베딩 모델을 사용하여 임베딩 생성 (모델의 임베딩 성능에 따라 검색능력이 많이 좌우 됨.)
# Chroma 벡터 스토어 구축
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="chroma_store_new_v2" # 벡터스토어를 저장할 폴더이름 중복되면 안된다.
)

# 유료임베딩
# vectorstore = Chroma.from_documents(documents=splits,
#                                     embedding=OpenAIEmbeddings())

In [53]:
docs = vectorstore.similarity_search("감정")
print(len(docs))
print(docs[0].page_content)

4
같이 보기[편집]
감정이입
감정 대립
감정과 기억
감정과 문화
자존감
각주[편집]


↑ 네이버 국어사전

↑ 중용

↑ 청춘심리학-#4 부정적 감정의 진짜 모습에 주목하라

↑ 부정적인 감정을 똑똑하게 표출하는 것은 건강에 좋다

↑ 서울&-내 삶의 주인 되기 자기 이해와 자기 긍정, 행복을 위한 필수조건



여성가족부-자녀연령별 육아정보-자녀의 감정을 다루어주기
vte심리학
역사
심리학자
연구 분야
감정
생물심리학
임상심리학
인지심리학
인지 신경과학
비교심리학
비판심리학
문화심리학
발달심리학
진화심리학
실험심리학
개인심리학
해방심리학
수리심리학
매체심리학
약물심리학
신경심리학
수행심리학
성격심리학
생리심리학
정치심리학
긍정심리학
심리언어학
정신병리학
정신물리학
심리생리학
정성적 심리 연구
정량적 심리 연구
사회심리학
이론심리학
교육심리학
군중심리학
스포츠심리학
응용 분야
심리 실험
임상심리학
상담심리학
교육심리학
법정심리학
건강심리학
산업 및 조직 심리학
법심리학
산업 건강심리학
관계심리학
학교심리학
스포츠심리학
음향심리학
체제심리학
심리철학
시각심리학
접근 방법
분석심리학
행동주의
인지주의
인지 행동 치료
기술심리학
실존주의 상담
가족 치료
인지 정서 행동 치료
여성주의 상담
게슈탈트 치료
인본주의심리학
초심리학
이야기 치료
정신분석학
정신 역동 치료
초개인심리학
주요 심리학자
버러스 프레더릭 스키너
장 피아제
지그문트 프로이트
오토 랑크
멜라니 클라인
앨버트 반두라
레온 페스팅거
로이 샤퍼
칼 로저스
스탠리 샤흐터
닐 엘가 밀러
에드워드 손다이크
에이브러햄 매슬로
고던 올포트
에릭 에릭슨
한스 아이젠크
윌프레드 비용
윌리엄 제임스
데이비드 맥클랜드
앨버트 엘리스
아론 벡
레이몬드 캐텔
존 B. 왓슨
쿠르트 르빈
도널드 올딩 헤브
조지 밀러
클라크 헐
제롬 케이건
카를 융
이반 파블로프
앙드레 그린
알프레트 아들러

전거 통제: 국가 프랑스BnF 데이터




이 글은 심리학에 관한 토막글입니다. 여러분의 지식으로 알차게 문서를 완성해 갑시다.


In [ ]:
# sudo apt install curl
# curl -fsSL https://ollama.com/install.sh | sh

# https://ollama.com/search ollama 모델들 검색후 사용할 모델 정하기

# 터미널에서 실행
# ollama serve & ollama pull gemma3:4b & ollama pull nomic-embed-text

In [60]:
import ollama
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import Optional


# Prompt 템플릿 정의
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# 7. Ollama Llama3 모델 설정
# Ollama의 LLM을 LangChain과 통합하기 위해 커스텀 LLM 클래스를 정의합니다.
from langchain.llms.base import LLM
from typing import Optional

class OllamaLLM(LLM):
    model_name: str = "gemma3:4b"
    temperature: float = 0.0

    def _call(self, prompt: str, stop: Optional[list] = None) -> str:
        response = ollama.chat(model=self.model_name, messages=[{'role': 'user', 'content': prompt}])
        return response['message']['content']

    @property
    def _llm_type(self):
        return "ollama"

# LLM 인스턴스 생성
llm = OllamaLLM(model_name="gemma3:4b", temperature=0.0) # temperature 가 작으면 모델의 대답 다양성이 적고, 높으면 대답을 다양하게 내뱉음


# Retriever 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # 상위 5개의 관련 문서 검색

# LLMChain 설정
llm_chain = LLMChain(prompt=prompt, llm=llm)


def rag_chain(question):
    # 관련 문서 검색
    retrieved_docs = retriever.get_relevant_documents(question)

    # 관련있는 문서의 내용을 뉴라인 기준으로 concatenate 함 이걸 모델한테 context로 전달해서 답변을 가져올 것이다.
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)


    # Prompt에 질문과 컨텍스트를 전달하여 답변 생성
    answer = llm_chain.run({"context": formatted_context, "question": question})
    return answer

# 인터랙티브하게 질문하고 응답 받기
print("질문을 입력하세요. 종료하려면 'exit'을 입력하세요.")

while True:
    question = input("질문: ")
    if question.lower() == 'exit':
        print("프로그램을 종료합니다.")
        break
    answer = rag_chain(question)
    print(f"답변: {answer}\n")

/tmp/ipykernel_168029/1553611022.py:45: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


질문을 입력하세요. 종료하려면 'exit'을 입력하세요.


/tmp/ipykernel_168029/1553611022.py:50: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)
/tmp/ipykernel_168029/1553611022.py:57: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = llm_chain.run({"context": formatted_context, "question": question})


답변: This question doesn't appear to be directly answerable based on the provided text. The text is a framework for building a Wikipedia-style entry on "Emotion." It lists topics, related terms, and provides a basic structure. 

Therefore, a response to "안녕" (Hello) isn’t present in the given context.

답변: 제공된 문맥을 바탕으로 "감정"에 대한 정보를 정리하면 다음과 같습니다.

**감정 - 위키백과, 우리 모두의 백과사전**

*   **개요:** 감정은 인간의 정신적, 생리적 반응으로, 특정 사건이나 자극에 대한 주관적인 경험입니다.
*   **언어:** 71개 언어로 제공됩니다.
*   **주요 내용:**
    *   **역사:**  (문서 내에 상세 정보는 없음)
    *   **관련 한자어:** (문서 내에 상세 정보는 없음)
    *   **긍정/부정 감정 표현:** (문서 내에 상세 정보는 없음)
    *   **슬픔:** (문서 내에 상세 정보는 없음)
    *   **정서:** (문서 내에 상세 정보는 없음)
    *   **같이 보기:** (감정이입, 감정 대립, 감정과 기억, 감정과 문화, 자존감)
    *   **각주:**  네이버 국어사전, 중용, 청춘심리학, 여성가족부

**추가 정보:**

*   문맥은 "감정"이라는 주제에 대한 심리학적 탐구를 위한 개요를 제공합니다.
*   다양한 심리학적 이론(예: 감정이입, 감정 대립, 감정과 기억 등)을 다루는 것을 암시합니다.
*   "감정" 관련 다양한 자료(사전, 논문, 육아 정보 등)를 참고할 수 있도록 합니다.

답변: 슬픔은 감정 중에서 특히 중요하게 다루어지는 감정으로, 개인이 심리적으로 부정적인 감정들을 해소하는 카타르시스적 역

In [ ]:
# RAG 코드를 쓰고 싶다면,

# 임베딩 모델 선정
# llm 모델 선정
# PDF 만들기